In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, RNN

In [2]:
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [4]:
decoded_review = ' '.join([reverse_word_index.get(i-3,'?') for i in x_train[1]])
decoded_review

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [5]:
y_train[1]

0

In [6]:
from tensorflow.keras.layers import SimpleRNN
max_features = 10000

In [7]:
max_len=500
x_train = sequence.pad_sequences(x_train,maxlen=max_len)
x_test = sequence.pad_sequences(x_test,maxlen=max_len)

In [8]:
model = Sequential()
model.add(Embedding(max_features,128,input_length=max_len))
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [12]:
history=model.fit(
    x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 153s 241ms/step - accuracy: 0.5658 - loss: 5798105.5000 - val_accuracy: 0.6078 - val_loss: 0.6389
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 196s 231ms/step - accuracy: 0.6485 - loss: 532.9954 - val_accuracy: 0.6874 - val_loss: 0.5918
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 213s 249ms/step - accuracy: 0.7696 - loss: 0.5264 - val_accuracy: 0.7152 - val_loss: 0.5571
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 198s 244ms/step - accuracy: 0.8264 - loss: 0.4542 - val_accuracy: 0.7270 - val_loss: 0.5408
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 147s 235ms/step - accuracy: 0.8557 - loss: 0.3995 - val_accuracy: 0.7326 - val_loss: 0.5345
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 204s 239ms/step - accuracy: 0.8773 - loss: 0.3486 - val_accuracy: 0.7376 - val_loss: 0.5352
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 203s 241ms/step - accuracy: 0.8956 - loss: 0.3052 - val_accuracy: 0.7442 - val_loss: 0.5400
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 201s 239ms/step - accuracy: 

In [13]:
model.save("imdb_model.h5")